In [ ]:
import pandas as pd
import os

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ece8953073f417635d1a3e3a6bab4011ff40edcf871d65875c0471c48a6ccc03
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.3 MB/s 
time: 751 µs (started: 2022-12-22 20:09:30 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 46 s (started: 2022-12-22 20:09:30 +00:00)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
import plotly.express as px
from pyspark.sql.window import Window

time: 547 ms (started: 2022-12-22 20:10:16 +00:00)


In [ ]:
scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.2.1'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.0'
]
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.jars.packages", ",".join(packages))\
        .getOrCreate()

time: 37.7 s (started: 2022-12-22 20:10:16 +00:00)


In [ ]:
spark

time: 1.33 s (started: 2022-12-22 20:10:54 +00:00)


In [ ]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.3.1/kafka_2.12-3.3.1.tgz

time: 1.33 s (started: 2022-12-22 20:13:01 +00:00)


In [ ]:
!tar -xzf kafka_2.12-3.3.1.tgz

time: 1.35 s (started: 2022-12-22 20:13:03 +00:00)


In [ ]:
!./kafka_2.12-3.3.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.3.1/config/zookeeper.properties
!./kafka_2.12-3.3.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.3.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running
time: 13.5 s (started: 2022-12-22 20:13:06 +00:00)


In [ ]:
def write_kafka(topic_name,file_path):
  df = spark.read.option("header",True).options(inferSchema='True').csv(file_path)
  os.system(f"./kafka_2.12-3.3.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic {topic_name}")
  df.select(F.to_json(F.struct("*")).alias("value"))\
  .selectExpr("CAST(value AS STRING)")\
  .write\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "127.0.0.1:9092")\
  .option("topic", topic_name)\
  .save()
  return df.schema


time: 1.96 ms (started: 2022-12-22 20:13:04 +00:00)


In [ ]:
def read_kafka(topic_name,schema):
  df1 = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", topic_name) \
  .load()
  df1 = df1.withColumn("value", F.col("value").cast(StringType()))

  info_dataframe = df1.select(
          F.from_json(F.col("value"), schema).alias("sample"), "timestamp"
      )
  info_dataframe = info_dataframe.select("sample.*")
  return info_dataframe

time: 3.43 ms (started: 2022-12-22 20:13:20 +00:00)


In [ ]:
#Data write and data in kafka

file_path = "/content/drive/MyDrive/fitbitData/dailyActivity_merged.csv"
topic_name = "activity_daily"
schema = write_kafka(topic_name,file_path)
userActivity = read_kafka(topic_name,schema)

time: 16.6 s (started: 2022-12-22 20:13:20 +00:00)


In [ ]:
userActivity.printSchema()

root
 |-- Id: long (nullable = true)
 |-- ActivityDate: string (nullable = true)
 |-- TotalSteps: integer (nullable = true)
 |-- TotalDistance: double (nullable = true)
 |-- TrackerDistance: double (nullable = true)
 |-- LoggedActivitiesDistance: double (nullable = true)
 |-- VeryActiveDistance: double (nullable = true)
 |-- ModeratelyActiveDistance: double (nullable = true)
 |-- LightActiveDistance: double (nullable = true)
 |-- SedentaryActiveDistance: double (nullable = true)
 |-- VeryActiveMinutes: integer (nullable = true)
 |-- FairlyActiveMinutes: integer (nullable = true)
 |-- LightlyActiveMinutes: integer (nullable = true)
 |-- SedentaryMinutes: integer (nullable = true)
 |-- Calories: integer (nullable = true)

time: 24.1 ms (started: 2022-12-22 20:20:17 +00:00)


In [ ]:
userActivity.show()

+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+
|        Id|ActivityDate|TotalSteps|   TotalDistance| TrackerDistance|LoggedActivitiesDistance|VeryActiveDistance|ModeratelyActiveDistance|LightActiveDistance|SedentaryActiveDistance|VeryActiveMinutes|FairlyActiveMinutes|LightlyActiveMinutes|SedentaryMinutes|Calories|
+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+
|1503960366|   4/12/2016|     13162|             8.5|             8.5|                     0.0|  1.87999999523163|       0.550000011920929|   6.05999994277954|                    0.0|          

In [ ]:
userActivity.groupBy("Id").agg(F.sum("TotalSteps"))

DataFrame[Id: bigint, sum(TotalSteps): bigint]

time: 25.7 ms (started: 2022-12-15 22:46:38 +00:00)


In [ ]:
df = userActivity.coalesce(10)

df.groupBy("Id").agg(F.sum("TotalSteps"))

DataFrame[Id: bigint, sum(TotalSteps): bigint]

time: 26.7 ms (started: 2022-12-15 22:46:39 +00:00)


In [ ]:
pandasDf = userActivity.toPandas()

fig = px.scatter(pandasDf,x = "TotalSteps",y="Calories",color="TotalDistance")
fig2 = px.scatter(pandasDf,x="TotalDistance",y = "Calories")
fig.show()
fig2.show()

In [ ]:
#Analyzing the trend between Total steps and the number of calories


grp_data = userActivity.groupBy('Id').agg(avg('TotalSteps').alias('Steps'),avg('Calories').alias('Calories'))
pd_data = grp_data.toPandas()
px.scatter(pd_data,x ='Steps',y='Calories').show() #SWARNA - UPDATE : WAY TO SHOW THE ID IN THE CHART

In [ ]:
#We are going to analyze how the users of fitbit are walking through the tracked days.
walking_data = userActivity.groupBy('ActivityDate').agg(avg('TotalSteps').alias('TotalSteps')).orderBy('ActivityDate')
pd_walking_data = walking_data.toPandas()
pd_walking_data['datetime'] = pd.to_datetime(pd_walking_data.ActivityDate)
pd_walking_data.sort_values('datetime',inplace=True)
walking_fig = px.line(pd_walking_data, x = 'ActivityDate', y= 'TotalSteps')
walking_fig.show()

In [ ]:
#Data write and data in kafka

file_path = "/content/drive/MyDrive/fitbitData/heartrate_seconds_merged.csv"
topic_name = "heartrate_seconds"
schema = write_kafka(topic_name,file_path)
heartRate = read_kafka(topic_name,schema)

In [ ]:
heartRate.show()

+----------+--------------------+-----+
|        Id|                Time|Value|
+----------+--------------------+-----+
|2022484408|4/12/2016 7:21:00 AM|   97|
|2022484408|4/12/2016 7:21:05 AM|  102|
|2022484408|4/12/2016 7:21:10 AM|  105|
|2022484408|4/12/2016 7:21:20 AM|  103|
|2022484408|4/12/2016 7:21:25 AM|  101|
|2022484408|4/12/2016 7:22:05 AM|   95|
|2022484408|4/12/2016 7:22:10 AM|   91|
|2022484408|4/12/2016 7:22:15 AM|   93|
|2022484408|4/12/2016 7:22:20 AM|   94|
|2022484408|4/12/2016 7:22:25 AM|   93|
|2022484408|4/12/2016 7:22:35 AM|   92|
|2022484408|4/12/2016 7:22:40 AM|   89|
|2022484408|4/12/2016 7:22:50 AM|   83|
|2022484408|4/12/2016 7:22:55 AM|   61|
|2022484408|4/12/2016 7:23:00 AM|   60|
|2022484408|4/12/2016 7:23:10 AM|   61|
|2022484408|4/12/2016 7:23:25 AM|   61|
|2022484408|4/12/2016 7:23:30 AM|   57|
|2022484408|4/12/2016 7:23:40 AM|   54|
|2022484408|4/12/2016 7:23:50 AM|   55|
+----------+--------------------+-----+
only showing top 20 rows



In [ ]:
#Extracting Time from the Timestamp column
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
heartRate = heartRate.withColumn("heart_time",date_format(to_timestamp(col("Time"),"mm/dd/yyyy hh:mm:ss aa"),"HH:mm:ss"))

#Extracting Date from the TimeStamp column
heartRate = heartRate.withColumn("Date",to_date(heartRate.Time, "mm/dd/yyyy HH:mm:ss"))

heartRate.show()

time: 2.24 ms (started: 2022-12-15 22:11:34 +00:00)


In [ ]:
heartRate = heartRate.withColumn("Hour", hour("heart_time"))

In [ ]:
heartRate.show()

+----------+--------------------+-----+----------+----------+----+
|        Id|                Time|Value|heart_time|      Date|Hour|
+----------+--------------------+-----+----------+----------+----+
|2022484408|4/12/2016 7:21:00 AM|   97|  07:21:00|2016-01-12|   7|
|2022484408|4/12/2016 7:21:05 AM|  102|  07:21:05|2016-01-12|   7|
|2022484408|4/12/2016 7:21:10 AM|  105|  07:21:10|2016-01-12|   7|
|2022484408|4/12/2016 7:21:20 AM|  103|  07:21:20|2016-01-12|   7|
|2022484408|4/12/2016 7:21:25 AM|  101|  07:21:25|2016-01-12|   7|
|2022484408|4/12/2016 7:22:05 AM|   95|  07:22:05|2016-01-12|   7|
|2022484408|4/12/2016 7:22:10 AM|   91|  07:22:10|2016-01-12|   7|
|2022484408|4/12/2016 7:22:15 AM|   93|  07:22:15|2016-01-12|   7|
|2022484408|4/12/2016 7:22:20 AM|   94|  07:22:20|2016-01-12|   7|
|2022484408|4/12/2016 7:22:25 AM|   93|  07:22:25|2016-01-12|   7|
|2022484408|4/12/2016 7:22:35 AM|   92|  07:22:35|2016-01-12|   7|
|2022484408|4/12/2016 7:22:40 AM|   89|  07:22:40|2016-01-12| 

In [ ]:
#Get the average heart rate per hour
avg_heart_rate = heartRate.groupBy('Id','Date','Hour').agg(avg('Value')).orderBy(['Hour','Id'])
# avg_heart_rate.show()

avg_heart_rate_hour = avg_heart_rate.groupBy('Hour','Id').agg(avg('avg(Value)')).orderBy(['Id','Hour'])
# avg_heart_rate_hour.show()
#Average heart_rate trend in each hour for all people
avg_heart_rate_general = avg_heart_rate_hour.groupBy("Hour").agg(avg('avg(avg(Value))').alias('Heart Rate')).orderBy("Hour")

#Convert to pandas Df
avg_heart_rate_general = avg_heart_rate_general.toPandas()

#Plotting the trend in the heart rate variations
fig = px.line(avg_heart_rate_general,x = 'Hour', y="Heart Rate")
fig.show()

In [ ]:
#Calorie Analysis
file_path = "/content/drive/MyDrive/fitbitData/hourlyCalories_merged.csv"
topic_name = "hourly_calories"
schema = write_kafka(topic_name,file_path)
calorie_df = read_kafka(topic_name,schema)

calorie_df = calorie_df.distinct()
calorie_df = calorie_df.na.drop('any')
calorie_df = calorie_df.withColumn('ActivityHour', from_unixtime(unix_timestamp(col(('ActivityHour')), 'MM/dd/yyyy hh:mm:ss a'),'YYYY-MM-dd HH:mm:ss'))
calorie_df.show()

+----------+-------------------+--------+
|        Id|       ActivityHour|Calories|
+----------+-------------------+--------+
|1503960366|2016-04-21 04:00:00|      47|
|1624580081|2016-04-14 18:00:00|      78|
|1624580081|2016-04-22 18:00:00|      51|
|1644430081|2016-04-13 17:00:00|     141|
|1644430081|2016-05-02 06:00:00|      84|
|1644430081|2016-05-05 17:00:00|     135|
|1844505072|2016-04-14 22:00:00|      80|
|1844505072|2016-04-20 07:00:00|      56|
|1844505072|2016-04-28 23:00:00|      88|
|1844505072|2016-04-30 12:00:00|     109|
|1844505072|2016-05-01 02:00:00|     117|
|1844505072|2016-05-11 14:00:00|      56|
|1927972279|2016-04-20 10:00:00|      86|
|1927972279|2016-05-02 09:00:00|      86|
|1927972279|2016-05-09 10:00:00|      86|
|1927972279|2016-05-10 22:00:00|      86|
|1927972279|2016-05-12 14:00:00|      86|
|2022484408|2016-04-12 18:00:00|     104|
|2022484408|2016-04-14 01:00:00|      62|
|2022484408|2016-05-06 07:00:00|      62|
+----------+-------------------+--

In [ ]:
calorie_df.show()

+----------+-------------------+--------+
|        Id|       ActivityHour|Calories|
+----------+-------------------+--------+
|1503960366|2016-04-21 04:00:00|      47|
|1624580081|2016-04-14 18:00:00|      78|
|1624580081|2016-04-22 18:00:00|      51|
|1644430081|2016-04-13 17:00:00|     141|
|1644430081|2016-05-02 06:00:00|      84|
|1644430081|2016-05-05 17:00:00|     135|
|1844505072|2016-04-14 22:00:00|      80|
|1844505072|2016-04-20 07:00:00|      56|
|1844505072|2016-04-28 23:00:00|      88|
|1844505072|2016-04-30 12:00:00|     109|
|1844505072|2016-05-01 02:00:00|     117|
|1844505072|2016-05-11 14:00:00|      56|
|1927972279|2016-04-20 10:00:00|      86|
|1927972279|2016-05-02 09:00:00|      86|
|1927972279|2016-05-09 10:00:00|      86|
|1927972279|2016-05-10 22:00:00|      86|
|1927972279|2016-05-12 14:00:00|      86|
|2022484408|2016-04-12 18:00:00|     104|
|2022484408|2016-04-14 01:00:00|      62|
|2022484408|2016-05-06 07:00:00|      62|
+----------+-------------------+--

In [ ]:
#Create a Separate date and time column in the same dataframe.
calorie_df = calorie_df.withColumn("Date",date_format(col("ActivityHour"),"yyyy-MM-dd"))
calorie_df = calorie_df.withColumn("Hour", hour(col("ActivityHour")))

#Now first group by ID, Hour and Date.
calorie_avg = calorie_df.groupBy('Id','Hour','Date').agg(avg('Calories').alias('Calories')).orderBy(['Id','Hour'])

#Now we will group by the Hour and ID.

calorie_avg_day = calorie_avg.groupBy('Id','Hour').agg(avg('Calories').alias('Calories')).orderBy(['Id','Hour']);

#The avg calories per hour in general.

calorie_avg_general = calorie_avg_day.groupBy('Hour').agg(avg('Calories').alias('Calories')).orderBy('Hour');
calorie_avg_general.show()


+----+------------------+
|Hour|          Calories|
+----+------------------+
|   0| 71.49556666709059|
|   1| 69.87763406617688|
|   2|  68.9527333129113|
|   3| 67.53634496348623|
|   4| 68.07395757951932|
|   5| 80.45515831344531|
|   6| 86.01867732302594|
|   7| 94.99401992829787|
|   8|104.73544453704498|
|   9|105.96581931180701|
|  10|109.76317938957473|
|  11|110.47060364757805|
|  12|115.94943380749265|
|  13| 114.5317810276285|
|  14|114.04263071824612|
|  15|105.46888114781197|
|  16|112.13676720725262|
|  17|120.18222293839104|
|  18|122.22549941675206|
|  19|119.76594556408303|
+----+------------------+
only showing top 20 rows



In [ ]:
def categorize(hour):
  if hour >=0 and hour <6:
    zone = 'Night'
  elif hour>=6 and hour <12:
    zone = 'Morning'
  elif hour>=12 and hour <18:
    zone = 'Afternoon'
  else:
    zone = 'Evening'
  return zone

zone_function = udf(categorize,StringType())
calorie_avg_general = calorie_avg_general.withColumn("Zone", zone_function(calorie_avg_general.Hour))
calorie_avg_general.show()

+----+------------------+---------+
|Hour|          Calories|     Zone|
+----+------------------+---------+
|   0| 71.49556666709059|    Night|
|   1| 69.87763406617688|    Night|
|   2|  68.9527333129113|    Night|
|   3| 67.53634496348623|    Night|
|   4| 68.07395757951932|    Night|
|   5| 80.45515831344531|    Night|
|   6| 86.01867732302594|  Morning|
|   7| 94.99401992829787|  Morning|
|   8|104.73544453704498|  Morning|
|   9|105.96581931180701|  Morning|
|  10|109.76317938957473|  Morning|
|  11|110.47060364757805|  Morning|
|  12|115.94943380749265|Afternoon|
|  13| 114.5317810276285|Afternoon|
|  14|114.04263071824612|Afternoon|
|  15|105.46888114781197|Afternoon|
|  16|112.13676720725262|Afternoon|
|  17|120.18222293839104|Afternoon|
|  18|122.22549941675206|  Evening|
|  19|119.76594556408303|  Evening|
+----+------------------+---------+
only showing top 20 rows



In [ ]:
df_calorie = calorie_avg_general.toPandas()

fig = px.pie(df_calorie,values = 'Calories',names='Zone')
fig2 = px.line(df_calorie,x = 'Hour', y="Calories")
fig.show()
fig2.show()

In [ ]:

file_path = "/content/drive/MyDrive/fitbitData/minuteIntensitiesNarrow_merged.csv"
topic_name = "minute_intensities"
schema = write_kafka(topic_name,file_path)
intensity_df = read_kafka(topic_name,schema)

intensity_df = intensity_df.distinct()
intensity_df = intensity_df.na.drop('any')
intensity_df = intensity_df.dropDuplicates(['Id','ActivityMinute'])


#Now we need to convert it to 24 hour format. And then separate Date and hour. Then categorize the intensities and add the different parts of days as zones.
intensity_df = intensity_df.withColumn('ActivityMinute', from_unixtime(unix_timestamp(col(('ActivityMinute')), 'MM/dd/yyyy hh:mm:ss a'),'YYYY-MM-dd HH:mm:ss'))
intensity_df = intensity_df.withColumn("Date",date_format(col("ActivityMinute"),"yyyy-MM-dd"))
intensity_df = intensity_df.withColumn("Hour", hour(col("ActivityMinute")))
intensity_df = intensity_df.withColumn('Intensity_Desc', when(col('Intensity') == '0','Sedentary').when(col('Intensity') == '1','Light').when(col('Intensity') == '2','Moderate').when(col('Intensity') == '3','Very Active'))
intensity_df = intensity_df.withColumn("Zone", zone_function(intensity_df.Hour))
# intensity_df.show()


day_zone_intensity = intensity_df.groupBy('Id','Date','Zone','Intensity_Desc').agg(count('Id').alias('NumberOfMinutes')).orderBy('Id','Date','Zone','Intensity_Desc')

zone_intensity_avg = day_zone_intensity.groupBy('Zone','Intensity_Desc').agg(avg('NumberOfMinutes').alias('Average')).orderBy('Zone','Intensity_Desc')

zone_intensity_sum = day_zone_intensity.groupBy('Zone','Intensity_Desc').agg(sum('NumberOfMinutes').alias('Sum')).orderBy('Zone','Intensity_Desc')

zone_intensity_sum.show()

In [ ]:
intensity_df.show()

+----------+--------------------+---------+
|        Id|      ActivityMinute|Intensity|
+----------+--------------------+---------+
|1503960366|4/12/2016 12:00:0...|        0|
|1503960366|4/12/2016 12:01:0...|        0|
|1503960366|4/12/2016 12:02:0...|        0|
|1503960366|4/12/2016 12:03:0...|        0|
|1503960366|4/12/2016 12:04:0...|        0|
|1503960366|4/12/2016 12:05:0...|        0|
|1503960366|4/12/2016 12:06:0...|        0|
|1503960366|4/12/2016 12:07:0...|        0|
|1503960366|4/12/2016 12:08:0...|        0|
|1503960366|4/12/2016 12:09:0...|        0|
|1503960366|4/12/2016 12:10:0...|        0|
|1503960366|4/12/2016 12:11:0...|        0|
|1503960366|4/12/2016 12:12:0...|        0|
|1503960366|4/12/2016 12:13:0...|        0|
|1503960366|4/12/2016 12:14:0...|        0|
|1503960366|4/12/2016 12:15:0...|        0|
|1503960366|4/12/2016 12:16:0...|        0|
|1503960366|4/12/2016 12:17:0...|        0|
|1503960366|4/12/2016 12:18:0...|        0|
|1503960366|4/12/2016 12:19:0...

In [ ]:
intensity_df = spark.read.options(inferSchema='True').csv('/content/drive/MyDrive/fitbitData/minuteIntensitiesNarrow_merged.csv', header=True )
intensity_df = intensity_df.distinct()
intensity_df = intensity_df.na.drop('any')
intensity_df = intensity_df.dropDuplicates(['Id','ActivityMinute'])


#Now we need to convert it to 24 hour format. And then separate Date and hour. Then categorize the intensities and add the different parts of days as zones.
intensity_df = intensity_df.withColumn('ActivityMinute', from_unixtime(unix_timestamp(col(('ActivityMinute')), 'MM/dd/yyyy hh:mm:ss a'),'YYYY-MM-dd HH:mm:ss'))
intensity_df = intensity_df.withColumn("Date",date_format(col("ActivityMinute"),"yyyy-MM-dd"))
intensity_df = intensity_df.withColumn("Hour", hour(col("ActivityMinute")))
intensity_df = intensity_df.withColumn('Intensity_Desc', when(col('Intensity') == '0','Sedentary').when(col('Intensity') == '1','Light').when(col('Intensity') == '2','Moderate').when(col('Intensity') == '3','Very Active'))
intensity_df = intensity_df.withColumn("Zone", zone_function(intensity_df.Hour))
# intensity_df.show()


day_zone_intensity = intensity_df.groupBy('Id','Date','Zone','Intensity_Desc').agg(count('Id').alias('NumberOfMinutes')).orderBy('Id','Date','Zone','Intensity_Desc')

zone_intensity_avg = day_zone_intensity.groupBy('Zone','Intensity_Desc').agg(avg('NumberOfMinutes').alias('Average')).orderBy('Zone','Intensity_Desc')

zone_intensity_sum = day_zone_intensity.groupBy('Zone','Intensity_Desc').agg(sum('NumberOfMinutes').alias('Sum')).orderBy('Zone','Intensity_Desc')

zone_intensity_sum.show()

+---------+--------------+------+
|     Zone|Intensity_Desc|   Sum|
+---------+--------------+------+
|Afternoon|         Light| 70416|
|Afternoon|      Moderate|  5028|
|Afternoon|     Sedentary|246256|
|Afternoon|   Very Active|  7460|
|  Evening|         Light| 51273|
|  Evening|      Moderate|  4310|
|  Evening|     Sedentary|264895|
|  Evening|   Very Active|  5322|
|  Morning|         Light| 52939|
|  Morning|      Moderate|  3267|
|  Morning|     Sedentary|272656|
|  Morning|   Very Active|  5938|
|    Night|         Light|  6263|
|    Night|      Moderate|   144|
|    Night|     Sedentary|328295|
|    Night|   Very Active|  1118|
+---------+--------------+------+



In [ ]:
morning_minute_sum = zone_intensity_sum.where(col('Zone') == 'Morning').toPandas()
morning_chart = px.pie(morning_minute_sum,values='Sum',names = 'Intensity_Desc')
morning_chart.show()

afternoon_minute_sum = zone_intensity_sum.where(col('Zone') == 'Afternoon').toPandas()
afternoon_chart = px.pie(afternoon_minute_sum,values='Sum',names = 'Intensity_Desc')
afternoon_chart.show()

evening_minute_sum = zone_intensity_sum.where(col('Zone') == 'Evening').toPandas()
evening_chart = px.pie(evening_minute_sum,values='Sum',names = 'Intensity_Desc')
evening_chart.show()

night_minute_sum = zone_intensity_sum.where(col('Zone') == 'Night').toPandas()
night_chart = px.pie(night_minute_sum,values='Sum',names = 'Intensity_Desc')
night_chart.show()

In [ ]:

file_path = "/content/drive/MyDrive/fitbitData/sleepDay_merged.csv"
topic_name = "sleepDay"
schema = write_kafka(topic_name,file_path)
sleep_df = read_kafka(topic_name,schema)
sleep_df = sleep_df.distinct()
sleep_df = sleep_df.na.drop('any')
sleep_df.show()

+----------+--------------------+-----------------+------------------+--------------+
|        Id|            SleepDay|TotalSleepRecords|TotalMinutesAsleep|TotalTimeInBed|
+----------+--------------------+-----------------+------------------+--------------+
|3977333714|5/8/2016 12:00:00 AM|                2|               259|           456|
|1503960366|4/29/2016 12:00:0...|                1|               341|           354|
|4388161847|4/28/2016 12:00:0...|                1|               428|           458|
|4445114986|4/30/2016 12:00:0...|                1|               322|           353|
|6117666160|5/5/2016 12:00:00 AM|                1|               392|           415|
|6962181067|4/26/2016 12:00:0...|                1|               441|           446|
|2026352035|4/12/2016 12:00:0...|                1|               503|           546|
|2347167796|4/26/2016 12:00:0...|                1|               436|           490|
|8378563200|5/5/2016 12:00:00 AM|                1|   

In [ ]:
def get_diff(col1,col2):
  return col1-col2

sleep_df = sleep_df.withColumn('TimeToFallSleep',get_diff(col('TotalTimeInBed'),col('TotalMinutesAsleep')))
sleep_df.show()

+----------+--------------------+-----------------+------------------+--------------+---------------+
|        Id|            SleepDay|TotalSleepRecords|TotalMinutesAsleep|TotalTimeInBed|TimeToFallSleep|
+----------+--------------------+-----------------+------------------+--------------+---------------+
|3977333714|5/8/2016 12:00:00 AM|                2|               259|           456|            197|
|1503960366|4/29/2016 12:00:0...|                1|               341|           354|             13|
|4388161847|4/28/2016 12:00:0...|                1|               428|           458|             30|
|4445114986|4/30/2016 12:00:0...|                1|               322|           353|             31|
|6117666160|5/5/2016 12:00:00 AM|                1|               392|           415|             23|
|6962181067|4/26/2016 12:00:0...|                1|               441|           446|              5|
|2026352035|4/12/2016 12:00:0...|                1|               503|           5

In [ ]:
#Calculate users average time to fall asleep
avg_TimeToFallSleep = sleep_df.groupBy("Id").agg(avg('TimeToFallSleep'))
avg_TimeToFallSleep.show()

+----------+--------------------+
|        Id|avg(TimeToFallSleep)|
+----------+--------------------+
|8378563200|  40.806451612903224|
|4702921684|   20.74074074074074|
|8792009665|  18.133333333333333|
|2026352035|  31.464285714285715|
|5553957443|   42.38709677419355|
|2320127002|                 8.0|
|8053475328|   4.666666666666667|
|4558609924|                12.4|
|6962181067|  18.129032258064516|
|4445114986|  31.642857142857142|
|6775888955|  19.333333333333332|
|7007744171|                 3.0|
|7086361926|  13.291666666666666|
|1644430081|                52.0|
|6117666160|   31.38888888888889|
|4020332650|              30.375|
|1844505072|               309.0|
|3977333714|               167.5|
|1503960366|               22.92|
|5577150313|  28.615384615384617|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# SWARNA - UPDATE : ADD CHART FOR ABOVE

In [ ]:
# CHIRAG
file_path = "/content/drive/MyDrive/fitbitData/dailySteps_merged.csv"
topic_name = "dailyStepsData"
schema = write_kafka(topic_name,file_path)
dailyStepsData = read_kafka(topic_name,schema)
dailyStepsData.createOrReplaceTempView("dailyStepsData")

time: 6.24 s (started: 2022-12-22 22:18:42 +00:00)


In [ ]:
spark.sql("SELECT * from dailyStepsData").show()

+----------+-----------+---------+
|        Id|ActivityDay|StepTotal|
+----------+-----------+---------+
|1503960366|  4/12/2016|    13162|
|1503960366|  4/13/2016|    10735|
|1503960366|  4/14/2016|    10460|
|1503960366|  4/15/2016|     9762|
|1503960366|  4/16/2016|    12669|
|1503960366|  4/17/2016|     9705|
|1503960366|  4/18/2016|    13019|
|1503960366|  4/19/2016|    15506|
|1503960366|  4/20/2016|    10544|
|1503960366|  4/21/2016|     9819|
|1503960366|  4/22/2016|    12764|
|1503960366|  4/23/2016|    14371|
|1503960366|  4/24/2016|    10039|
|1503960366|  4/25/2016|    15355|
|1503960366|  4/26/2016|    13755|
|1503960366|  4/27/2016|    18134|
|1503960366|  4/28/2016|    13154|
|1503960366|  4/29/2016|    11181|
|1503960366|  4/30/2016|    14673|
|1503960366|   5/1/2016|    10602|
+----------+-----------+---------+
only showing top 20 rows

time: 580 ms (started: 2022-12-22 22:18:49 +00:00)


In [ ]:
dailyStepsData = dailyStepsData.withColumn("Date",to_date(dailyStepsData.ActivityDay, "MM/dd/yyyy"))

dailyStepsData = dailyStepsData.withColumn("Weekday",dayofweek(dailyStepsData.Date))

dailyStepsData = dailyStepsData.withColumn("weekday_str",date_format(dailyStepsData.Date,"EEEE"))

time: 76.1 ms (started: 2022-12-22 22:18:51 +00:00)


In [ ]:
dailyStepsData.show()

Py4JJavaError: ignored

time: 906 ms (started: 2022-12-22 22:18:53 +00:00)


In [ ]:
dailyStepsData.write.csv("dailySteps.csv")

Py4JJavaError: ignored

time: 1.13 s (started: 2022-12-22 22:18:55 +00:00)


In [ ]:
dailyStepsData.coalesce(10).write.parquet("dailySteps1")

AnalysisException: ignored

time: 276 ms (started: 2022-12-15 22:43:50 +00:00)


In [ ]:
dailyStepsData = spark.read.csv("dailySteps.csv")

AnalysisException: ignored

time: 135 ms (started: 2022-12-22 22:19:03 +00:00)


In [ ]:
dailyStepsDataParquet = spark.read.parquet("dailySteps1")

AnalysisException: ignored

time: 125 ms (started: 2022-12-22 22:18:59 +00:00)


In [ ]:
dailyStepsData.groupBy(dailyStepsData._c5).agg(avg(dailyStepsData._c2).alias("avgSteps"))

DataFrame[_c5: string, avgSteps: double]

time: 21.3 ms (started: 2022-12-15 22:16:56 +00:00)


In [ ]:
dailyStepsDataParquet.groupBy(dailyStepsDataParquet.weekday_str).agg(avg(dailyStepsDataParquet.StepTotal).alias("avgSteps"))

DataFrame[weekday_str: string, avgSteps: double]

time: 28.4 ms (started: 2022-12-15 22:16:57 +00:00)


In [ ]:
avg_steps = dailyStepsData.groupBy(dailyStepsData.weekday_str).agg(avg(dailyStepsData.StepTotal).alias("avgSteps"))

avg_steps = avg_steps.select("*",
               F.when(F.col("weekday_str") == "Monday", 1)
               .when(F.col("weekday_str") == "Tuesday", 2)
               .when(F.col("weekday_str") == "Wednesday", 3)
               .when(F.col("weekday_str") == "Thursday", 4)
               .when(F.col("weekday_str") == "Friday", 5)
               .when(F.col("weekday_str") == "Saturday", 6)
               .when(F.col("weekday_str") == "Sunday", 7)
               .otherwise(0)
               .alias("weekday_order")
              )
avg_steps_ordered = avg_steps.orderBy(F.col("weekday_order").asc())
avg_steps_ordered.show()

avg_steps_pandas = avg_steps_ordered.toPandas()
fig = px.line(avg_steps_pandas, x="weekday_str", y="avgSteps", title='Average steps by users per day of the week')
fig.show()

+-----------+-----------------+-------------+
|weekday_str|         avgSteps|weekday_order|
+-----------+-----------------+-------------+
|     Monday|7780.866666666667|            1|
|    Tuesday|8125.006578947368|            2|
|  Wednesday|7559.373333333333|            3|
|   Thursday|7405.836734693878|            4|
|     Friday|7448.230158730159|            5|
|   Saturday|8152.975806451613|            6|
|     Sunday|6933.231404958678|            7|
+-----------+-----------------+-------------+



In [ ]:
dailyStepsData = dailyStepsData.withColumn("StepTotal",dailyStepsData["StepTotal"].cast(IntegerType()))
grouped_df = dailyStepsData.groupBy("Id", "weekday_str").sum("StepTotal")
pandas_df = grouped_df.toPandas()
fig = px.bar(data_frame=pandas_df, x="weekday_str", y="sum(StepTotal)", color = "Id")
fig.show()

In [ ]:
dailyStepsPandas = dailyStepsData.toPandas()
fig = px.scatter(dailyStepsPandas,x="weekday_str",y="StepTotal")
fig.show()

In [ ]:
file_path = "/content/drive/MyDrive/fitbitData/sleepDay_merged.csv"
topic_name = "sleepDay"
schema = write_kafka(topic_name,file_path)
sleepDf = read_kafka(topic_name,schema)

In [ ]:
sleepDf = sleepDf.withColumn("Date",to_date(sleepDf.SleepDay, "MM/dd/yyyy HH:mm:ss"))

sleepDf = sleepDf.withColumn("Weekday",dayofweek(sleepDf.Date))

sleepDf = sleepDf.withColumn("weekday_str",date_format(sleepDf.Date,"EEEE"))


In [ ]:
windowSpec  = Window.partitionBy("Id","weekday_str").orderBy("TotalMinutesAsleep")

sleepDf.groupBy(sleepDf.weekday_str).agg(F.collect_list(F.col("TotalMinutesAsleep")).alias("totalminutes"))\
.withColumn("len",F.size(F.col("totalminutes"))).show()

+-----------+--------------------+---+
|weekday_str|        totalminutes|len|
+-----------+--------------------+---+
|  Wednesday|[384, 360, 285, 3...|132|
|    Tuesday|[327, 304, 245, 2...|130|
|     Friday|[412, 341, 334, 1...|114|
|   Thursday|[325, 366, 247, 1...|130|
|   Saturday|[340, 361, 404, 3...|116|
|     Monday|[277, 277, 338, 7...| 94|
|     Sunday|[700, 430, 369, 5...|110|
+-----------+--------------------+---+



In [ ]:
avg_sleep = sleepDf.groupBy(sleepDf.weekday_str).agg(avg(sleepDf.TotalMinutesAsleep).alias("avgSleep"))
avg_sleep.show()

avg_sleep = avg_sleep.select("*",
               F.when(F.col("weekday_str") == "Monday", 1)
               .when(F.col("weekday_str") == "Tuesday", 2)
               .when(F.col("weekday_str") == "Wednesday", 3)
               .when(F.col("weekday_str") == "Thursday", 4)
               .when(F.col("weekday_str") == "Friday", 5)
               .when(F.col("weekday_str") == "Saturday", 6)
               .when(F.col("weekday_str") == "Sunday", 7)
               .otherwise(0)
               .alias("weekday_order")
              )
avg_sleep_ordered = avg_sleep.orderBy(F.col("weekday_order").asc())
# avg_sleep_ordered.show()

avg_sleep_pandas = avg_sleep_ordered.toPandas()
fig = px.line(avg_sleep_pandas, x="weekday_str", y="avgSleep", title='Average sleep by users per day of the week')
fig.show()

NameError: ignored

time: 74.5 ms (started: 2022-12-22 22:18:27 +00:00)


In [ ]:
file_path = "/content/drive/MyDrive/fitbitData/minuteIntensitiesNarrow_merged.csv"
topic_name = "minuteIntensitiesNarrow"
schema = write_kafka(topic_name,file_path)
minuteIntensities = read_kafka(topic_name,schema)

minuteIntensities_df = minuteIntensities.withColumn('Intensity_Desc', F.when(col('Intensity') == '0','Sedentary').when(col('Intensity') == '1','Light').when(col('Intensity') == '2','Moderate').when(col('Intensity') == '3','Very Active'))
minuteIntensityDf = minuteIntensities_df.withColumn("Date",to_timestamp(minuteIntensities_df.ActivityMinute, "MM/dd/yyyy HH:mm:ss"))
minuteIntensityDf = minuteIntensityDf.withColumn("date_column",to_date(minuteIntensities_df.ActivityMinute, "MM/dd/yyyy"))

time: 37.7 s (started: 2022-12-15 00:31:43 +00:00)


In [ ]:
minuteIntensityDf = minuteIntensityDf.withColumn('Date1', F.from_unixtime(F.unix_timestamp(F.col(('ActivityMinute')), 'MM/dd/yyyy hh:mm:ss a'),'YYYY-MM-dd HH:mm:ss'))
minuteIntensityDf = minuteIntensityDf.withColumn("hour_column", F.hour(F.col("Date1")))

time: 91.3 ms (started: 2022-12-15 00:32:21 +00:00)


In [ ]:
groupedByMinuteIntensity = minuteIntensityDf.groupBy(minuteIntensityDf.Id,minuteIntensityDf.date_column,minuteIntensityDf.hour_column,minuteIntensityDf.Intensity_Desc).agg(F.count("Intensity_Desc").alias("count_mins"))

groupedByMinuteIntensity = groupedByMinuteIntensity.groupBy(groupedByMinuteIntensity.hour_column,groupedByMinuteIntensity.Intensity_Desc).agg(F.avg(F.col("count_mins")).alias("hour_mins"))
groupedByMinuteIntensity.show()
totalIntensityMinsGroupedByHour = groupedByMinuteIntensity.groupBy(groupedByMinuteIntensity.hour_column).agg(F.sum(groupedByMinuteIntensity.hour_mins))
totalIntensityMinsGroupedByHour.show()

+-----------+--------------+------------------+
|hour_column|Intensity_Desc|         hour_mins|
+-----------+--------------+------------------+
|          3|         Light|2.9361702127659575|
|         22|      Moderate|10.255813953488373|
|          9|   Very Active|15.228571428571428|
|          0|   Very Active|3.4285714285714284|
|         18|   Very Active|15.309734513274336|
|         15|      Moderate| 8.264705882352942|
|         17|     Sedentary| 43.15631929046563|
|          8|         Light|14.097938144329897|
|         21|     Sedentary| 49.97120708748616|
|         11|   Very Active|11.833333333333334|
|          5|     Sedentary| 57.29077253218884|
|          4|     Sedentary| 59.36695278969957|
|         12|      Moderate| 8.080357142857142|
|          2|   Very Active|              19.0|
|         16|         Light| 16.41917808219178|
|          6|      Moderate|               9.0|
|         13|     Sedentary| 45.14176663031625|
|         23|     Sedentary|55.854767184

In [ ]:
groupedByMinuteIntensity.count()

92

time: 20.8 s (started: 2022-12-15 00:33:13 +00:00)


In [ ]:
totalIntensityMinsGroupedByHour.count()

24

time: 18.5 s (started: 2022-12-15 00:33:34 +00:00)


In [ ]:
joinedIntensityTable = groupedByMinuteIntensity.join(totalIntensityMinsGroupedByHour,"hour_column","leftouter")

time: 225 ms (started: 2022-12-15 00:34:01 +00:00)


In [ ]:
joinedIntensityTable = groupedByMinuteIntensity.join(F.broadcast(totalIntensityMinsGroupedByHour),"hour_column","leftouter")

time: 154 ms (started: 2022-12-15 00:39:23 +00:00)


In [ ]:
pandas_df_intensities = joinedIntensityTable.toPandas()
fig = px.bar(pandas_df_intensities, x="hour_column", y="hour_mins", color="Intensity_Desc")
fig.show()

In [ ]:
file_path = "/content/drive/MyDrive/fitbitData/hourlyCalories_merged.csv"
topic_name = "hourlyCalories"
schema = write_kafka(topic_name,file_path)
calorie_df = read_kafka(topic_name,schema)

calorie_df = calorie_df.distinct()
calorie_df = calorie_df.na.drop('any')
calorie_df = calorie_df.withColumn('ActivityHour', from_unixtime(unix_timestamp(col(('ActivityHour')), 'MM/dd/yyyy hh:mm:ss a'),'YYYY-MM-dd HH:mm:ss'))
# calorie_df.show()

calorie_df = calorie_df.withColumn("Date",date_format(col("ActivityHour"),"yyyy-MM-dd"))
calorie_df = calorie_df.withColumn("Hour", hour(col("ActivityHour")))
# calorie_df.show()

#Now first group by ID, Hour and Date.
calorie_avg = calorie_df.groupBy('Id','Hour','Date').agg(avg('Calories').alias('Calories')).orderBy(['Id','Hour'])
# calorie_avg.show()

#Now we will group by the Hour and ID.

calorie_avg_day = calorie_avg.groupBy('Id','Hour').agg(avg('Calories').alias('Calories')).orderBy(['Id','Hour']);
# calorie_avg_day.show()

#The avg calories per hour in general.

calorie_avg_general = calorie_avg_day.groupBy('Hour').agg(avg('Calories').alias('Calories')).orderBy('Hour');
# calorie_avg_general.show()

calorie_avg_general_pandas = calorie_avg_general.toPandas()
fig = px.bar(calorie_avg_general_pandas,x = 'Hour',y='Calories')
fig.show()

In [ ]:
userActivity.show()

+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+
|        Id|ActivityDate|TotalSteps|   TotalDistance| TrackerDistance|LoggedActivitiesDistance|VeryActiveDistance|ModeratelyActiveDistance|LightActiveDistance|SedentaryActiveDistance|VeryActiveMinutes|FairlyActiveMinutes|LightlyActiveMinutes|SedentaryMinutes|Calories|
+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+
|1503960366|   4/12/2016|     13162|             8.5|             8.5|                     0.0|  1.87999999523163|       0.550000011920929|   6.05999994277954|                    0.0|          

In [ ]:
userActivity = userActivity.withColumn("TotalActiveMinutes",userActivity.LightlyActiveMinutes + userActivity.FairlyActiveMinutes  + userActivity.VeryActiveMinutes)

In [ ]:
userActivity.show()

+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+------------------+
|        Id|ActivityDate|TotalSteps|   TotalDistance| TrackerDistance|LoggedActivitiesDistance|VeryActiveDistance|ModeratelyActiveDistance|LightActiveDistance|SedentaryActiveDistance|VeryActiveMinutes|FairlyActiveMinutes|LightlyActiveMinutes|SedentaryMinutes|Calories|TotalActiveMinutes|
+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+------------------+
|1503960366|   4/12/2016|     13162|             8.5|             8.5|                     0.0|  1.87999999523163|       0.5500000119209

In [ ]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 4.9 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


time: 11.7 s (started: 2022-12-15 20:54:20 +00:00)


In [ ]:
userActivity.show()

+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+
|        Id|ActivityDate|TotalSteps|   TotalDistance| TrackerDistance|LoggedActivitiesDistance|VeryActiveDistance|ModeratelyActiveDistance|LightActiveDistance|SedentaryActiveDistance|VeryActiveMinutes|FairlyActiveMinutes|LightlyActiveMinutes|SedentaryMinutes|Calories|
+----------+------------+----------+----------------+----------------+------------------------+------------------+------------------------+-------------------+-----------------------+-----------------+-------------------+--------------------+----------------+--------+
|1503960366|   4/12/2016|     13162|             8.5|             8.5|                     0.0|  1.87999999523163|       0.550000011920929|   6.05999994277954|                    0.0|          

In [ ]:
import six
corr_dict = {}
for i in userActivity.columns:
    if not( isinstance(userActivity.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to TotalActiveMinutes for ", i, userActivity.stat.corr('TotalActiveMinutes',i))
        corr_dict[i] = userActivity.stat.corr('TotalActiveMinutes',i)

Correlation to TotalActiveMinutes for  Id 0.0017491501117729903
Correlation to TotalActiveMinutes for  TotalSteps 0.7724292872482617
Correlation to TotalActiveMinutes for  TotalDistance 0.7226050847114914
Correlation to TotalActiveMinutes for  TrackerDistance 0.721093983057151
Correlation to TotalActiveMinutes for  LoggedActivitiesDistance 0.16363857974389542
Correlation to TotalActiveMinutes for  VeryActiveDistance 0.3113719527386354
Correlation to TotalActiveMinutes for  ModeratelyActiveDistance 0.36155150442588085
Correlation to TotalActiveMinutes for  LightActiveDistance 0.8719706335606014
Correlation to TotalActiveMinutes for  SedentaryActiveDistance 0.10989109775119893
Correlation to TotalActiveMinutes for  VeryActiveMinutes 0.3675449809410307
Correlation to TotalActiveMinutes for  FairlyActiveMinutes 0.38181612799451004
Correlation to TotalActiveMinutes for  LightlyActiveMinutes 0.9349496847225564
Correlation to TotalActiveMinutes for  SedentaryMinutes -0.4752583816612084
Correl

In [ ]:
for key,val in corr_dict.items():
  if val > 0.5 :
    print(key,val)

TotalSteps 0.7724292872482617
TotalDistance 0.7226050847114914
TrackerDistance 0.721093983057151
LightActiveDistance 0.8719706335606014
LightlyActiveMinutes 0.9349496847225564
TotalActiveMinutes 1.0


In [ ]:
activity = userActivity.select("TotalSteps","TotalDistance","LightActiveDistance","LightlyActiveMinutes","TotalActiveMinutes")

In [ ]:
activity.show()

+----------+----------------+-------------------+--------------------+------------------+
|TotalSteps|   TotalDistance|LightActiveDistance|LightlyActiveMinutes|TotalActiveMinutes|
+----------+----------------+-------------------+--------------------+------------------+
|     13162|             8.5|   6.05999994277954|                 328|               366|
|     10735|6.96999979019165|   4.71000003814697|                 217|               257|
|     10460|6.73999977111816|   3.91000008583069|                 181|               222|
|      9762|6.28000020980835|   2.82999992370605|                 209|               272|
|     12669|8.15999984741211|   5.03999996185303|                 221|               267|
|      9705|6.48000001907349|   2.50999999046326|                 164|               222|
|     13019|8.59000015258789|   4.71000003814697|                 233|               291|
|     15506|9.88000011444092|   5.03000020980835|                 264|               345|
|     1054

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ["TotalSteps","TotalDistance","LightActiveDistance","LightlyActiveMinutes"], outputCol = 'features')
vhouse_df = vectorAssembler.transform(activity)
vhouse_df = vhouse_df.select(['features', 'TotalActiveMinutes'])
vhouse_df.show(3)

+--------------------+------------------+
|            features|TotalActiveMinutes|
+--------------------+------------------+
|[13162.0,8.5,6.05...|               366|
|[10735.0,6.969999...|               257|
|[10460.0,6.739999...|               222|
+--------------------+------------------+
only showing top 3 rows



In [ ]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
train_df.count()

1295

time: 3.12 s (started: 2022-12-15 22:09:25 +00:00)


In [ ]:
test_df.count()

585

time: 1.48 s (started: 2022-12-15 22:09:33 +00:00)


In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='TotalActiveMinutes', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))


Coefficients: [0.006942555035860162,2.31815999784426,-6.999135061931641,0.9176347173966595]
Intercept: 8.106622056888188


In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 24.176544
r2: 0.960318


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","TotalActiveMinutes","features").show(5)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="TotalActiveMinutes",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-----------------+------------------+---------+
|       prediction|TotalActiveMinutes| features|
+-----------------+------------------+---------+
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
+-----------------+------------------+---------+
only showing top 5 rows

R Squared (R2) on test data = 0.962957


In [ ]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 23.5335


In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","TotalActiveMinutes","features").show()

+-----------------+------------------+---------+
|       prediction|TotalActiveMinutes| features|
+-----------------+------------------+---------+
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188|                 0|(4,[],[])|
|8.106622056888188| 

In [ ]:
predictions

DataFrame[features: vector, TotalActiveMinutes: int, prediction: double]